# 01 — Exploratory Data Analysis (EDA)
## HVAC Market Analysis — Metropolitan France (96 departments)

**Objective**: Explore and understand the HVAC ML dataset before modeling.

**Plan**:
1. Load and overview the dataset
2. Missing values analysis
3. Distribution of target variables
4. Time series and trends
5. Seasonality of the HVAC market
6. Department comparison
7. Correlations between variables
8. New features: SITADEL (construction) and INSEE Filosofi (socioeconomic)
9. Conclusions for modeling

In [ ]:
# ============================================================
# IMPORTS AND CONFIGURATION
# ============================================================
import sys
sys.path.insert(0, '..')  # To import project modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11
sns.set_palette('Set2')

# Full DataFrame display
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', '{:.2f}'.format)

print('Imports OK')

---
## 1. Load and overview the dataset

The ML-ready dataset contains data aggregated by **month x department**:
- 96 metropolitan French departments
- DPE v2 period: July 2021 onwards
- Target variables: heat pump installations, air conditioning, total DPE count

In [ ]:
# ============================================================
# 1.1 — Load the ML-ready dataset
# ============================================================
# This dataset is produced by: python -m src.pipeline merge
# Grain: month x department (96 depts x N months)

df = pd.read_csv('../data/features/hvac_ml_dataset.csv')

print(f'ML-ready dataset: {df.shape[0]} rows x {df.shape[1]} columns')
print(f'Period: {df["date_id"].min()} -> {df["date_id"].max()}')
print(f'Departments: {len(df["dept"].unique())} unique')
print(f'\nColumn types:')
print(df.dtypes.value_counts())

In [ ]:
# ============================================================
# 1.2 — Preview the first rows
# ============================================================
df.head(10)

In [ ]:
# ============================================================
# 1.3 — Global descriptive statistics
# ============================================================
df.describe().T.round(2)

---
## 1.4 — Data Quality Audit: NaN Detection & Traceability

**Objective**: Systematically detect all NaN in the dataset and trace their origin back to the source data.

This audit ensures:
- Every NaN is explained (collection gap, feature engineering lag, missing source)
- Department coverage is verified (96/96 expected)
- Temporal coverage is complete (no missing months)
- Any column with >5% NaN is flagged for investigation

In [ ]:
# ============================================================
# 1.4a — Per-column NaN Count & Percentage
# ============================================================
nan_count = df.isna().sum()
nan_pct = (df.isna().mean() * 100).round(2)
nan_report = pd.DataFrame({
    'NaN_count': nan_count,
    'NaN_pct': nan_pct,
    'dtype': df.dtypes
}).sort_values('NaN_pct', ascending=False)

# Flag severity levels
nan_report['severity'] = nan_report['NaN_pct'].apply(
    lambda x: '🔴 CRITICAL (>50%)' if x > 50
    else '🟡 WARNING (>5%)' if x > 5
    else '🟢 OK' if x > 0
    else '✅ CLEAN'
)

print(f"{'='*70}")
print(f"  NaN AUDIT — ML-ready dataset ({df.shape[0]} rows × {df.shape[1]} cols)")
print(f"{'='*70}")
total_nan = nan_count.sum()
total_cells = df.shape[0] * df.shape[1]
print(f"Total NaN: {total_nan:,} / {total_cells:,} cells ({100*total_nan/total_cells:.2f}%)")
print(f"Columns with NaN: {(nan_pct > 0).sum()} / {len(nan_pct)}")
print()

# Show problematic columns
problematic = nan_report[nan_report['NaN_pct'] > 0]
if len(problematic) > 0:
    print("Columns with NaN (sorted by severity):")
    display(problematic[['NaN_count', 'NaN_pct', 'severity']])
else:
    print("✅ No NaN in the dataset — all clean!")

In [ ]:
# ============================================================
# 1.4b — NaN Provenance: Trace each NaN back to its source
# ============================================================
# Classify columns by data source to identify where NaN originate

SOURCE_PATTERNS = {
    'Weather (Open-Meteo)': ['temp_', 'hdd', 'cdd', 'precip', 'wind', 'pac_inefficient',
                              'elevation', 'extreme_cold', 'extreme_hot', 'frost_days'],
    'DPE (ADEME)': ['nb_dpe', 'nb_installations', 'pct_pac', 'pct_clim', 'pct_classe'],
    'INSEE (economic)': ['confiance_menages', 'climat_affaires', 'chomage', 'permis_construire',
                          'prix_immobilier', 'taux_pauvrete', 'revenu_median', 'densite_pop'],
    'Eurostat (industry)': ['ipi_', 'eurostat'],
    'SITADEL (construction)': ['sitadel', 'logements_autorises', 'logements_commences'],
    'Feature eng. (lag/rolling)': ['lag_', 'rolling_', 'diff_', 'trend_', 'interact_',
                                     'ratio_', 'seasonal_', 'momentum_'],
}

def classify_column(col_name):
    """Identify the data source of a column by pattern matching."""
    for source, patterns in SOURCE_PATTERNS.items():
        if any(p in col_name.lower() for p in patterns):
            return source
    return 'Other / Identifier'

nan_by_source = {}
for col in df.columns:
    source = classify_column(col)
    if source not in nan_by_source:
        nan_by_source[source] = {'cols_total': 0, 'cols_with_nan': 0, 'nan_cells': 0}
    nan_by_source[source]['cols_total'] += 1
    if df[col].isna().sum() > 0:
        nan_by_source[source]['cols_with_nan'] += 1
        nan_by_source[source]['nan_cells'] += df[col].isna().sum()

print(f"{'='*70}")
print(f"  NaN PROVENANCE — By data source")
print(f"{'='*70}")
source_df = pd.DataFrame(nan_by_source).T
source_df['nan_pct_of_total'] = (source_df['nan_cells'] / total_nan * 100).round(1) if total_nan > 0 else 0
display(source_df)

# Detailed breakdown: show affected columns per source
print("\n--- Detailed breakdown: affected columns per source ---")
for source, patterns in SOURCE_PATTERNS.items():
    affected = [
        col for col in df.columns
        if classify_column(col) == source and df[col].isna().sum() > 0
    ]
    if affected:
        print(f"\n📌 {source} ({len(affected)} column(s) with NaN):")
        for col in affected:
            pct = 100 * df[col].isna().mean()
            print(f"   {col:40s} → {pct:5.1f}% NaN ({df[col].isna().sum()} rows)")

In [ ]:
# ============================================================
# 1.4c — Department Coverage Check (96 metropolitan expected)
# ============================================================
EXPECTED_DEPTS = {
    '01','02','03','04','05','06','07','08','09','10',
    '11','12','13','14','15','16','17','18','19',
    '21','22','23','24','25','26','27','28','29',
    '2A','2B',
    '30','31','32','33','34','35','36','37','38','39',
    '40','41','42','43','44','45','46','47','48','49',
    '50','51','52','53','54','55','56','57','58','59',
    '60','61','62','63','64','65','66','67','68','69',
    '70','71','72','73','74','75','76','77','78','79',
    '80','81','82','83','84','85','86','87','88','89',
    '90','91','92','93','94','95',
}

actual_depts = set(df['dept'].astype(str).unique())
missing_depts = EXPECTED_DEPTS - actual_depts
extra_depts = actual_depts - EXPECTED_DEPTS

print(f"{'='*70}")
print(f"  DEPARTMENT COVERAGE CHECK")
print(f"{'='*70}")
print(f"Expected: {len(EXPECTED_DEPTS)} departments")
print(f"Found:    {len(actual_depts)} departments")

if not missing_depts:
    print("✅ All 96 metropolitan departments are present!")
else:
    print(f"🔴 MISSING {len(missing_depts)} departments: {sorted(missing_depts)}")

if extra_depts:
    print(f"⚠️  Extra departments (unexpected): {sorted(extra_depts)}")

# Check rows per department (should be uniform)
rows_per_dept = df.groupby('dept').size()
print(f"\nRows per department: min={rows_per_dept.min()}, max={rows_per_dept.max()}, "
      f"mean={rows_per_dept.mean():.0f}")
if rows_per_dept.min() != rows_per_dept.max():
    print("⚠️  Non-uniform row count — some months may be missing for certain departments")

In [ ]:
# ============================================================
# 1.4d — Temporal Coverage Check (no missing months)
# ============================================================
dates = sorted(df['date_id'].unique())
print(f"{'='*70}")
print(f"  TEMPORAL COVERAGE CHECK")
print(f"{'='*70}")
print(f"Date range: {dates[0]} → {dates[-1]}")
print(f"Unique months: {len(dates)}")

# Check for gaps between consecutive months
gaps = []
for i in range(1, len(dates)):
    prev_y, prev_m = divmod(dates[i-1], 100) if dates[i-1] > 100 else (dates[i-1], 1)
    curr_y, curr_m = divmod(dates[i], 100) if dates[i] > 100 else (dates[i], 1)
    expected_next = prev_y * 100 + prev_m + 1 if prev_m < 12 else (prev_y + 1) * 100 + 1
    if dates[i] != expected_next:
        gaps.append((dates[i-1], dates[i]))

if not gaps:
    print("✅ No temporal gaps — continuous month coverage!")
else:
    print(f"🔴 {len(gaps)} temporal gap(s) detected:")
    for before, after in gaps:
        print(f"   Gap between {before} and {after}")

# Check per-department temporal completeness
dept_date_counts = df.groupby('dept')['date_id'].nunique()
incomplete_depts = dept_date_counts[dept_date_counts < len(dates)]
if len(incomplete_depts) == 0:
    print(f"✅ All departments have {len(dates)} months (complete)")
else:
    print(f"⚠️  {len(incomplete_depts)} department(s) with fewer than {len(dates)} months:")
    for dept, count in incomplete_depts.items():
        print(f"   Dept {dept}: {count}/{len(dates)} months")

In [ ]:
# ============================================================
# 1.4e — Raw Source Verification (cross-check with source files)
# ============================================================
import os
from pathlib import Path

RAW_DIR = Path('../data/raw')
PROCESSED_DIR = Path('../data/processed')

SOURCES_TO_CHECK = {
    'Weather': ('weather/weather_france.csv', 'dept'),
    'DPE': ('dpe', None),  # multi-file directory
    'INSEE': ('insee', None),
    'Eurostat': ('eurostat', None),
    'SITADEL': ('sitadel', None),
}

print(f"{'='*70}")
print(f"  RAW SOURCE FILE VERIFICATION")
print(f"{'='*70}")

for source_name, (path, dept_col) in SOURCES_TO_CHECK.items():
    full_path = RAW_DIR / path
    if full_path.is_file():
        try:
            src_df = pd.read_csv(full_path, dtype={dept_col: str} if dept_col else None)
            n_rows = len(src_df)
            n_depts = src_df[dept_col].nunique() if dept_col and dept_col in src_df.columns else 'N/A'
            status = '✅' if (dept_col is None or n_depts == 96) else f'🔴 {n_depts}/96 depts'
            print(f"  {source_name:12s} | {str(full_path):45s} | {n_rows:>7,} rows | Depts: {status}")
        except Exception as e:
            print(f"  {source_name:12s} | ERROR reading: {e}")
    elif full_path.is_dir():
        files = list(full_path.glob('*.csv'))
        total_rows = 0
        for f in files:
            try:
                total_rows += len(pd.read_csv(f))
            except Exception:
                pass
        print(f"  {source_name:12s} | {str(full_path):45s} | {len(files)} CSV files, {total_rows:>7,} total rows")
    else:
        print(f"  {source_name:12s} | ❌ NOT FOUND: {full_path}")

print()
print("Root cause reference:")
print("  - Weather NaN → API rate limiting during Open-Meteo collection (429)")
print("  - Lag/rolling NaN → Structural: first N months lack sufficient history")
print("  - INSEE/Eurostat NaN → National indicators not yet published for recent months")
print("  - SITADEL NaN → Construction permits coverage gap")

---
## 2. Missing values analysis

NaN values can come from:
- Economic indicators not available for some months
- Months without DPE in a department
- Eurostat indicators not yet published
- SITADEL or reference data not available for some department/month combinations

In [ ]:
# ============================================================
# 2.1 — Count NaN per column
# ============================================================
null_pct = (df.isna().mean() * 100).sort_values(ascending=False)
null_cols = null_pct[null_pct > 0]

if len(null_cols) > 0:
    print(f'{len(null_cols)} columns with NaN:')
    for col, pct in null_cols.items():
        print(f'  {col:40s} : {pct:.1f}% ({df[col].isna().sum()} values)')
else:
    print('No NaN in the ML-ready dataset!')

In [ ]:
# ============================================================
# 2.2 — NaN heatmap (matrix visualization)
# ============================================================
fig, ax = plt.subplots(figsize=(16, 6))
sns.heatmap(
    df.select_dtypes(include=[np.number]).isna().T,
    cbar=False, cmap='Reds', yticklabels=True,
    ax=ax
)
ax.set_title('Missing values map (red = NaN)', fontsize=14)
ax.set_xlabel('Row index (month x dept)')
plt.tight_layout()
plt.show()

---
## 3. Distribution of target variables

Main target variables:
- **nb_installations_pac**: DPE mentioning a heat pump (proxy for heat pump sales)
- **nb_installations_clim**: DPE with air conditioning
- **nb_dpe_total**: total DPE volume (real estate activity indicator)
- **nb_dpe_classe_ab**: DPE class A or B (high-performance buildings)

In [ ]:
# ============================================================
# 3.1 — Histograms of the 4 target variables
# ============================================================
targets = ['nb_dpe_total', 'nb_installations_pac', 
           'nb_installations_clim', 'nb_dpe_classe_ab']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Distribution of target variables', fontsize=14)

for ax, col in zip(axes.flat, targets):
    ax.hist(df[col], bins=30, edgecolor='black', alpha=0.7, density=True, label='Histogram')
    df[col].plot(kind='kde', ax=ax, color='red', linewidth=2, label='KDE')
    
    mean_val = df[col].mean()
    median_val = df[col].median()
    ax.axvline(mean_val, color='blue', linestyle='--', label=f'Mean={mean_val:.0f}')
    ax.axvline(median_val, color='green', linestyle='--', label=f'Median={median_val:.0f}')
    
    ax.set_title(col)
    ax.legend(fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 3.2 — Boxplots of targets by department (top 20 by volume)
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Target distribution by department (Top 20)', fontsize=14)

for ax, col in zip(axes.flat, targets):
    # Sort departments by descending median, show top 20
    order = df.groupby('dept')[col].median().sort_values(ascending=False).head(20).index
    sns.boxplot(data=df[df['dept'].isin(order)], x='dept', y=col, order=order, ax=ax)
    ax.set_title(col)
    ax.set_xlabel('Department')
    ax.tick_params(axis='x', rotation=45, labelsize=8)

plt.tight_layout()
plt.show()

---
## 4. Time series and trends

Examine monthly evolution of target variables to detect:
- **Long-term trends** (heat pump market growth?)
- **Seasonality** (installation peaks in winter/summer?)
- **Anomalies** (unusual months)

In [ ]:
# ============================================================
# 4.1 — Prepare time axis
# ============================================================
# Convert date_id (YYYYMM) to datetime for plots

df['date'] = pd.to_datetime(
    df['date_id'].astype(str).str[:4] + '-' + 
    df['date_id'].astype(str).str[4:] + '-01'
)

# Aggregate at national level (sum of all departments)
df_national = df.groupby('date').agg({
    'nb_dpe_total': 'sum',
    'nb_installations_pac': 'sum',
    'nb_installations_clim': 'sum',
    'nb_dpe_classe_ab': 'sum',
    'temp_mean': 'mean',
    'hdd_sum': 'sum',
    'cdd_sum': 'sum',
}).reset_index()

n_depts = df['dept'].nunique()
print(f'National time series: {len(df_national)} months (aggregated from {n_depts} departments)')

In [ ]:
# ============================================================
# 4.2 — Heat pump installations vs temperature (national aggregate)
# ============================================================
fig, ax1 = plt.subplots(figsize=(14, 6))

# Heat pump installations
ax1.plot(df_national['date'], df_national['nb_installations_pac'], 
         'b-o', markersize=4, label='Heat pump installations', linewidth=2)
ax1.fill_between(df_national['date'], df_national['nb_installations_pac'], alpha=0.1, color='blue')
ax1.set_ylabel('Nb heat pump installations (national)', color='blue', fontsize=12)

# Mean temperature overlay
ax2 = ax1.twinx()
ax2.plot(df_national['date'], df_national['temp_mean'], 
         'r--', alpha=0.7, label='Mean temp.', linewidth=1.5)
ax2.set_ylabel('Mean temperature (C)', color='red', fontsize=12)

ax1.set_title('Heat pump installations vs Temperature — Metropolitan France', fontsize=14)
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 4.3 — Time series for top 8 departments (by volume)
# ============================================================
top_depts = df.groupby('dept')['nb_installations_pac'].sum().nlargest(8).index

fig, axes = plt.subplots(2, 4, figsize=(18, 8), sharey=False)
fig.suptitle('Heat pump installations — Top 8 departments', fontsize=14)

for ax, dept in zip(axes.flat, top_depts):
    dept_data = df[df['dept'] == dept].sort_values('date')
    dept_name = dept_data['dept_name'].iloc[0] if 'dept_name' in dept_data.columns else dept
    
    ax.plot(dept_data['date'], dept_data['nb_installations_pac'], 
            '-o', markersize=2, linewidth=1)
    ax.set_title(f'{dept_name} ({dept})', fontsize=10)
    ax.tick_params(axis='x', rotation=45, labelsize=7)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 5. HVAC market seasonality

Business hypotheses:
- **Heating (heat pumps)**: installation peak in autumn (winter anticipation)
- **Air conditioning**: peak in spring/early summer (before heatwaves)
- **Total DPE**: linked to real estate transactions (spring peak)

In [ ]:
# ============================================================
# 5.1 — Seasonal boxplots (by month)
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Seasonality — Distribution by month', fontsize=14)

month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for ax, col in zip(axes.flat, targets):
    sns.boxplot(data=df, x='month', y=col, ax=ax)
    ax.set_xticklabels(month_labels, rotation=45)
    ax.set_title(col)
    ax.set_xlabel('')
    
    # Color the seasons
    for i in range(12):
        if i+1 in [10, 11, 12, 1, 2, 3]:  # Heating season
            ax.axvspan(i-0.5, i+0.5, alpha=0.05, color='blue')
        elif i+1 in [6, 7, 8, 9]:  # Cooling season
            ax.axvspan(i-0.5, i+0.5, alpha=0.05, color='red')

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 5.2 — Heatmap month x department (heat pump installations, top 15)
# ============================================================
top15_depts = df.groupby('dept_name' if 'dept_name' in df.columns else 'dept')[
    'nb_installations_pac'
].mean().nlargest(15).index

col_name = 'dept_name' if 'dept_name' in df.columns else 'dept'
df_top = df[df[col_name].isin(top15_depts)]

pivot_pac = df_top.pivot_table(
    values='nb_installations_pac', 
    index='month', 
    columns=col_name,
    aggfunc='mean'
)
pivot_pac.index = month_labels

fig, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(
    pivot_pac, annot=True, fmt='.0f', cmap='YlOrRd',
    linewidths=0.5, ax=ax
)
ax.set_title('Mean heat pump installations — Month x Department (Top 15)', fontsize=14)
ax.set_xlabel('Department')
ax.set_ylabel('Month')
plt.tight_layout()
plt.show()

---
## 6. Department comparison

In [ ]:
# ============================================================
# 6.1 — Mean volume per department (top 20)
# ============================================================
col_name = 'dept_name' if 'dept_name' in df.columns else 'dept'
dept_stats = df.groupby(col_name).agg({
    'nb_dpe_total': 'mean',
    'nb_installations_pac': 'mean',
    'nb_installations_clim': 'mean',
    'pct_pac': 'mean',
}).round(1).sort_values('nb_dpe_total', ascending=False)

print(f'Mean monthly statistics by department (top 20 / {len(dept_stats)} total):')
dept_stats.head(20)

In [ ]:
# ============================================================
# 6.2 — Comparative bar charts (top 15 departments)
# ============================================================
top15_stats = dept_stats.head(15)

fig, axes = plt.subplots(1, 3, figsize=(16, 6))
fig.suptitle('Department comparison — Monthly mean (Top 15)', fontsize=14)

for ax, col, title in zip(axes, 
    ['nb_dpe_total', 'nb_installations_pac', 'pct_pac'],
    ['Total DPE volume', 'Heat pump installations', 'Heat pump rate (%)']):
    
    top15_stats[col].plot(kind='barh', ax=ax, color='steelblue')
    ax.set_title(title)
    ax.set_xlabel(col)

plt.tight_layout()
plt.show()

---
## 7. Correlations between variables

Objective: identify the features most correlated with targets
to guide feature selection for modeling.

In [ ]:
# ============================================================
# 7.1 — Correlation matrix (base features)
# ============================================================
numeric_cols = df.select_dtypes(include=[np.number]).columns
exclude = ['date_id', 'latitude', 'longitude']
corr_cols = [c for c in numeric_cols if c not in exclude]

corr_matrix = df[corr_cols].corr()

fig, ax = plt.subplots(figsize=(16, 14))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
sns.heatmap(
    corr_matrix, mask=mask, annot=False, cmap='RdBu_r',
    center=0, vmin=-1, vmax=1, linewidths=0.5,
    ax=ax
)
ax.set_title('Correlation matrix — Base features', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 7.2 — Top correlations with target (nb_installations_pac)
# ============================================================
target = 'nb_installations_pac'

corr_target = corr_matrix[target].drop(target).abs().sort_values(ascending=False)

print(f'Top 15 correlations with {target}:')
print('-' * 50)
for feat, corr_val in corr_target.head(15).items():
    sign = '+' if corr_matrix.loc[feat, target] > 0 else '-'
    print(f'  {feat:40s} : {sign}{corr_val:.3f}')

In [ ]:
# ============================================================
# 7.3 — Scatter plots: key features vs target
# ============================================================
key_features = ['nb_dpe_total', 'temp_mean', 'hdd_sum', 
                'confiance_menages', 'ipi_hvac_c28', 'cdd_sum']
key_features = [f for f in key_features if f in df.columns]

n = len(key_features)
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle(f'Feature relationships -> {target}', fontsize=14)

for ax, feat in zip(axes.flat, key_features):
    ax.scatter(df[feat], df[target], alpha=0.3, s=10)
    
    # Trend line
    mask = df[[feat, target]].dropna().index
    if len(mask) > 5:
        z = np.polyfit(df.loc[mask, feat], df.loc[mask, target], 1)
        p = np.poly1d(z)
        x_range = np.linspace(df[feat].min(), df[feat].max(), 100)
        ax.plot(x_range, p(x_range), 'r-', linewidth=2, alpha=0.7)
    
    r = df[feat].corr(df[target])
    ax.set_title(f'{feat}\n(r = {r:.3f})', fontsize=10)
    ax.set_xlabel(feat)
    ax.set_ylabel(target)
    ax.grid(True, alpha=0.3)

for i in range(n, len(axes.flat)):
    axes.flat[i].set_visible(False)

plt.tight_layout()
plt.show()

---
## 9. Conclusions for modeling

### Observed data characteristics:
- **Dataset**: ~5376 rows (96 departments x 56 months), 60+ features
- **Zero NaN**: the merge pipeline produces a clean, complete dataset — no imputation bias
- **Target** (`nb_installations_pac`): right-skewed distribution with high inter-department variance (Ile-de-France departments >> rural areas)

### Key observations:

1. **Target distributions**: volumes vary greatly between departments (Paris/Ile-de-France >> rural departments). Mean ~107 installations/month/dept but with std ~100+ indicating huge variability. The model must capture this geographic effect.

2. **Seasonality**: visible seasonal pattern — heat pump installations peak in autumn (Oct-Nov, anticipation of winter) and dip in summer (Aug). Air conditioning peaks in spring/early summer (before heatwaves). Weather features (HDD/CDD) will be essential.

3. **Trend**: heat pump market shows growth over the 2021-2025 period, driven by MaPrimeRenov' incentives and energy transition policies. The `year` feature captures this dynamic.

4. **Correlations**: `nb_dpe_total` is strongly correlated with `nb_installations_pac` (volume effect: more DPE = more heat pump detections). `hdd_sum` shows positive correlation (cold weather -> more heating installations). Economic indicators (household confidence, business climate) add supplementary signal.

5. **New features**: SITADEL (construction permits) and INSEE reference (altitude, population density) provide department-level structural context. These features are integrated via `python -m src.pipeline merge` then `features`.

### Implications for ML:
- **Ridge Regression**: strong baseline candidate — L2 regularization handles correlated features (lags, rolling) well
- **LightGBM**: can capture non-linear interactions (weather x department, season x economic indicators)
- **Prophet**: adapted to the time series component with external regressors, but limited by small data per department (~36 months each)
- **LSTM**: exploratory only — insufficient data volume for deep learning

### Improvement suggestions:
- **Stationarity test** (ADF / KPSS) on national time series — confirm whether differencing is needed
- **Granger causality test** for economic indicators — quantify predictive lead of confidence indices
- **Violin plots** instead of boxplots for richer distribution view (shows multimodality)
- **Cross-correlation analysis** at different lags between weather and installations (optimal lag identification)

### Next steps:
- Notebook 02: ML modeling (Ridge, LightGBM, Prophet)
- Notebook 05: Feature review and SHAP analysis (column-level keep/drop decisions)

In [ ]:
# ============================================================
# Summary
# ============================================================
n_depts = df['dept'].nunique()
print('=' * 60)
print('EDA SUMMARY')
print('=' * 60)
print(f'Dataset: {df.shape[0]} rows x {df.shape[1]} columns')
print(f'Period: {df["date_id"].min()} -> {df["date_id"].max()}')
print(f'Departments: {n_depts}')
print(f'Total NaN: {df.isna().sum().sum()} ({(df.isna().mean().mean()*100):.1f}%)')
print(f'\nTarget variable (nb_installations_pac):')
print(f'  Mean: {df["nb_installations_pac"].mean():.1f}')
print(f'  Std: {df["nb_installations_pac"].std():.1f}')
print(f'  Min: {df["nb_installations_pac"].min()}')
print(f'  Max: {df["nb_installations_pac"].max()}')
print('=' * 60)

In [ ]:
# ============================================================
# 8 — New features: SITADEL & INSEE Filosofi reference
# ============================================================
# Check which new features are available in the dataset

# SITADEL (construction permits, per department)
sitadel_cols = [c for c in df.columns if c.startswith('nb_logements') or c.startswith('surface_autorisee')]

# INSEE Filosofi reference (static, per department)
ref_cols = [c for c in df.columns if c in [
    'revenu_median', 'prix_m2_median', 'nb_logements_total', 'pct_maisons'
]]

new_features = sitadel_cols + ref_cols

if new_features:
    print(f'New features found: {len(new_features)}')
    for feat in new_features:
        null_pct = df[feat].isna().mean() * 100
        r = df[feat].corr(df[target]) if df[feat].notna().sum() > 10 else float('nan')
        print(f'  {feat:35s} : NaN={null_pct:.1f}%, corr={r:+.3f}')
    
    # Scatter plots for reference features
    plot_feats = [f for f in new_features if f in df.columns and df[f].notna().sum() > 10]
    if plot_feats:
        n = min(len(plot_feats), 4)
        fig, axes = plt.subplots(1, n, figsize=(4 * n, 4))
        if n == 1:
            axes = [axes]
        fig.suptitle(f'New features vs {target}', fontsize=13)
        for ax, feat in zip(axes, plot_feats[:4]):
            ax.scatter(df[feat], df[target], alpha=0.3, s=10)
            r = df[feat].corr(df[target])
            ax.set_title(f'{feat}\n(r={r:+.3f})', fontsize=9)
            ax.set_xlabel(feat, fontsize=8)
            ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
else:
    print('No SITADEL or reference features found in the ML dataset.')
    print('These features are in hvac_features_dataset.csv (used by modeling notebooks).')